In [2]:
#Define the molecule
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

molecule = Molecule(
    geometry=[["He", [0.0, 0.0, 0.0]], ["He", [0.0, 0.512, 0.735]], ["H", [0.3, 0.418, 0.972]]], charge=3, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)
qubit_converter = QubitConverter(JordanWignerMapper())


#Define the solver

from qiskit_nature.algorithms import NumPyEigensolverFactory

numpy_solver = NumPyEigensolverFactory(use_default_filter_criterion=True)


#Define qEOM as solver


from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import GroundStateEigensolver, QEOM, VQEUCCFactory

# This first part sets the ground state solver
# see more about this part in the ground state calculation tutorial
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator_statevector"))
solver = VQEUCCFactory(quantum_instance=quantum_instance)
gsc = GroundStateEigensolver(qubit_converter, solver)

# The qEOM algorithm is simply instantiated with the chosen ground state solver
qeom_excited_states_calculation = QEOM(gsc, "sd")



#Calculation and results

from qiskit_nature.algorithms import ExcitedStatesEigensolver

numpy_excited_states_calculation = ExcitedStatesEigensolver(qubit_converter, numpy_solver)
numpy_results = numpy_excited_states_calculation.solve(es_problem)

qeom_results = qeom_excited_states_calculation.solve(es_problem)

print(numpy_results)
print("\n\n")
print(qeom_results)


#Use of the custom filter function to check consistently our results and only filter out states with incorrect number of particle (in this case the number of particle is 2).

import numpy as np


def filter_criterion(eigenstate, eigenvalue, aux_values):
    return np.isclose(aux_values[0][0], 2.0)


new_numpy_solver = NumPyEigensolverFactory(filter_criterion=filter_criterion)
new_numpy_excited_states_calculation = ExcitedStatesEigensolver(qubit_converter, new_numpy_solver)
new_numpy_results = new_numpy_excited_states_calculation.solve(es_problem)

print(new_numpy_results) 

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -7.50612286773
  - computed part:      -7.50612286773
~ Nuclear repulsion energy (Hartree): 6.013571831969
> Total ground state energy (Hartree): -1.49255103576
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -6.835337144897
> Total excited state energy (Hartree): -0.821765312928
  2: 
* Electronic excited state energy (Hartree): -5.733860403355
> Total excited state energy (Hartree): 0.279711428614
  3: 
* Electronic excited state energy (Hartree): -5.37902767491
> Total excited state energy (Hartree): 0.634544157059
  4: 
* Electronic excited state energy (Hartree): -4.831714892711
> Total excited state energy (Hartree): 1.181856939258
  5: 
* Electronic excited state energy (Hartree): -3.179423643027
> Total excited state energy (Hartree): 2.834148188942
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 2.000 S: 0.000